## Preprocessing

In [2]:
!pip install tensorflow

  Using cached tensorflow-2.15.0-cp39-cp39-win_amd64.whl (2.1 kB)
  Using cached tensorflow_intel-2.15.0-cp39-cp39-win_amd64.whl (300.8 MB)
  Using cached keras-2.15.0-py3-none-any.whl (1.7 MB)
  Using cached tensorflow_estimator-2.15.0-py2.py3-none-any.whl (441 kB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached gast-0.5.4-py3-none-any.whl (19 kB)
  Using cached absl_py-2.1.0-py3-none-any.whl (133 kB)
  Using cached flatbuffers-23.5.26-py2.py3-none-any.whl (26 kB)
  Using cached libclang-16.0.6-py2.py3-none-win_amd64.whl (24.4 MB)
  Using cached tensorboard-2.15.1-py3-none-any.whl (5.5 MB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached termcolor-2.4.0-py3-none-any.whl (7.7 kB)
  Using cached tensorflow_io_gcs_filesystem-0.31.0-cp39-cp39-win_amd64.whl (1.5 MB)
  Using cached ml_dtypes-0.2.0-cp39-cp39-win_amd64.whl (938 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached google_auth_oauthlib-1.2.0-py2.py3-non

In [6]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dla-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [7]:
# Drop the non-beneficial ID column 'NAME'
# Optimisation attempt 1 (Keep 'EIN' column)
application_df = application_df.drop(columns=['NAME'])
application_df.head()

,EIN,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [8]:
# Determine the number of unique values in each column.
application_df.nunique()

EIN                       34299
APPLICATION_TYPE             17
AFFILIATION                   6
CLASSIFICATION               71
USE_CASE                      5
ORGANIZATION                  4
STATUS                        2
INCOME_AMT                    9
SPECIAL_CONSIDERATIONS        2
ASK_AMT                    8747
IS_SUCCESSFUL                 2
dtype: int64

In [9]:
# Look at APPLICATION_TYPE value counts for binning
application_df['APPLICATION_TYPE'].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [10]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_type_list = application_df['APPLICATION_TYPE'].value_counts().to_dict()
application_types_to_replace = []
for i in application_type_list:
    if application_type_list[i] < 528:
        application_types_to_replace.append(i)
# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [11]:
# Look at CLASSIFICATION value counts for binning
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [12]:
# You may find it helpful to look at CLASSIFICATION value counts >1
application_df['CLASSIFICATION'].value_counts().loc[lambda x: x>1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [13]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classification_type_list = application_df['CLASSIFICATION'].value_counts().to_dict()
classifications_to_replace = []
for i in classification_type_list:
    if classification_type_list[i] < 1883:
        classifications_to_replace.append(i)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [14]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df)

In [16]:
# Split our preprocessed data into our features and target arrays
y = application_df["IS_SUCCESSFUL"].values
X = application_df.drop(columns="IS_SUCCESSFUL").values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42) 

In [17]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [38]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

# Optimisation attempt 2 (add more units to each hidden layer)
# First hidden layer
nn.add(tf.keras.layers.Dense(units=32, activation="relu", input_dim=44))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=32, activation="relu"))

# Third hidden layer
# Optimisation attempt 3 (add two more hidden layers)
nn.add(tf.keras.layers.Dense(units=32, activation="relu"))

# Fourth hidden layer
nn.add(tf.keras.layers.Dense(units=32, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_17 (Dense)            (None, 32)                1440      
                                                                 
 dense_18 (Dense)            (None, 32)                1056      
                                                                 
 dense_19 (Dense)            (None, 32)                1056      
                                                                 
 dense_20 (Dense)            (None, 32)                1056      
                                                                 
 dense_21 (Dense)            (None, 1)                 33        
                                                                 
Total params: 4641 (18.13 KB)
Trainable params: 4641 (18.13 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [39]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


In [40]:
# Train the model
# Optimisation attempt 4 (add more epochs)
fit_model = nn.fit(X_train_scaled, y_train, epochs=120)


Epoch 1/120
804/804 [==============================] - 2s 1ms/step - loss: 0.5704 - accuracy: 0.7214
Epoch 2/120
804/804 [==============================] - 1s 1ms/step - loss: 0.5518 - accuracy: 0.7303
Epoch 3/120
804/804 [==============================] - 1s 1ms/step - loss: 0.5476 - accuracy: 0.7311
Epoch 4/120
804/804 [==============================] - 1s 1ms/step - loss: 0.5457 - accuracy: 0.7323
Epoch 5/120
804/804 [==============================] - 1s 1ms/step - loss: 0.5433 - accuracy: 0.7348
Epoch 6/120
804/804 [==============================] - 1s 2ms/step - loss: 0.5419 - accuracy: 0.7341
Epoch 7/120
804/804 [==============================] - 1s 1ms/step - loss: 0.5408 - accuracy: 0.7361
Epoch 8/120
804/804 [==============================] - 1s 1ms/step - loss: 0.5395 - accuracy: 0.7371
Epoch 9/120
804/804 [==============================] - 1s 1ms/step - loss: 0.5388 - accuracy: 0.7363
Epoch 10/120
804/804 [==============================] - 1s 1ms/step - loss: 0.5380 - accura

804/804 [==============================] - 1s 1ms/step - loss: 0.5106 - accuracy: 0.7542
Epoch 82/120
804/804 [==============================] - 1s 1ms/step - loss: 0.5094 - accuracy: 0.7556
Epoch 83/120
804/804 [==============================] - 1s 1ms/step - loss: 0.5098 - accuracy: 0.7542
Epoch 84/120
804/804 [==============================] - 1s 1ms/step - loss: 0.5097 - accuracy: 0.7530
Epoch 85/120
804/804 [==============================] - 1s 1ms/step - loss: 0.5097 - accuracy: 0.7547
Epoch 86/120
804/804 [==============================] - 1s 1ms/step - loss: 0.5097 - accuracy: 0.7539
Epoch 87/120
804/804 [==============================] - 1s 1ms/step - loss: 0.5086 - accuracy: 0.7557
Epoch 88/120
804/804 [==============================] - 1s 1ms/step - loss: 0.5084 - accuracy: 0.7548
Epoch 89/120
804/804 [==============================] - 1s 1ms/step - loss: 0.5087 - accuracy: 0.7542
Epoch 90/120
804/804 [==============================] - 1s 1ms/step - loss: 0.5088 - accuracy: 

In [41]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5961 - accuracy: 0.7284 - 369ms/epoch - 1ms/step
Loss: 0.5961245894432068, Accuracy: 0.728396475315094


In [42]:
# Export our model to HDF5 file
nn.save_weights('AlphabetSoupCharity_Optimisation.h5')